In [3]:
!pip install transformers

In [4]:
!pip install torch

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers.data.processors.utils import InputExample
from torch import nn
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from transformers import glue_convert_examples_to_features,DistilBertTokenizer,DistilBertForSequenceClassification
from scipy.stats import ks_2samp
import pickle

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=200)

model.classifier.add_module('activation', nn.Linear(768, 200))
model.classifier.add_module('prediction', nn.Softmax(5))

model.classifier.load_state_dict(torch.load('Yelp Classifier.pt',map_location=torch.device('cpu')))
model.pre_classifier.load_state_dict(torch.load('Yelp Pre_Classifier.pt',map_location=torch.device('cpu')))

model.eval()
# print(model)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifie

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [6]:

adf = pd.read_csv('check.csv')
adf=adf.sample(500)

def get_features(df, text_col, label_col):
    l = [InputExample(guid=idx, text_a=df.loc[idx, text_col], label=df.loc[idx, label_col]) for idx, row in df.iterrows()]
    features = glue_convert_examples_to_features(examples=l, 
                                    tokenizer=tokenizer,
                                    max_length=300,
                                    label_list = df[label_col].values,
                                    output_mode='regression')
#     print(features[0].token_type_ids)
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_labels = torch.tensor([int(f.label-1) for f in features], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_attention_mask, all_labels)
    return dataset

def score(curr_dataset,ref_dataset):
    return 1e5/ks_2samp(ref_dataset,curr_dataset).statistic

test_dataset = get_features(adf, 'text', 'stars')

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
batch_predictions, batch_actual = [], []
with torch.no_grad():
    for k, (input_ids, attention_mask, labels) in enumerate(tqdm(test_dataloader, total=len(test_dataloader), desc='val batch')):
        feed_dict = {'input_ids': input_ids,
                     'attention_mask': attention_mask} 
        
        pred = model(**feed_dict)[0].cpu()
        batch_predictions.append(pred.numpy())
        batch_actual.append(labels)

c:\Python310\lib\site-packages\transformers\data\processors\glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


val batch:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
predictions = np.array([i for k in batch_predictions for i in k ])

predictions = np.argmax(predictions, axis=1)
actual = np.array([i for k in batch_actual for i in k ])

In [8]:
predictions

array([4, 2, 4, 1, 0, 4, 2, 4, 0, 4, 1, 0, 4, 4, 0, 3, 0, 4, 1, 0, 2, 3,
       0, 0, 4, 2, 2, 4, 0, 2, 4, 2, 1, 4, 3, 0, 1, 2, 4, 1, 1, 2, 4, 1,
       0, 2, 2, 1, 2, 4, 2, 3, 3, 2, 2, 2, 1, 1, 2, 2, 1, 4, 1, 2, 2, 1,
       1, 4, 4, 2, 0, 4, 1, 3, 1, 2, 1, 3, 3, 2, 4, 0, 3, 2, 4, 2, 3, 4,
       2, 2, 0, 4, 2, 0, 3, 4, 4, 3, 0, 0, 2, 0, 3, 0, 0, 1, 2, 3, 2, 2,
       1, 3, 2, 2, 2, 2, 1, 4, 2, 2, 3, 1, 3, 0, 4, 1, 4, 0, 2, 4, 2, 2,
       2, 2, 2, 1, 2, 3, 0, 0, 4, 1, 2, 2, 4, 1, 0, 2, 3, 4, 3, 1, 2, 3,
       4, 4, 2, 2, 1, 2, 0, 2, 3, 2, 2, 1, 3, 2, 0, 2, 3, 2, 2, 4, 2, 4,
       3, 0, 2, 3, 2, 0, 2, 2, 2, 2, 4, 2, 4, 0, 2, 4, 0, 2, 2, 4, 0, 4,
       4, 4, 4, 1, 0, 4, 2, 4, 2, 3, 2, 3, 1, 3, 0, 0, 2, 4, 0, 3, 4, 2,
       2, 2, 1, 0, 0, 0, 4, 4, 2, 4, 2, 2, 4, 3, 4, 2, 2, 0, 0, 1, 3, 2,
       4, 4, 2, 1, 1, 1, 3, 3, 1, 4, 4, 3, 0, 2, 1, 2, 1, 3, 1, 2, 1, 0,
       3, 0, 1, 0, 0, 3, 0, 3, 2, 4, 2, 0, 4, 2, 0, 0, 0, 3, 2, 2, 0, 1,
       2, 0, 3, 2, 2, 2, 2, 1, 2, 1, 1, 4, 1, 0, 2,

In [9]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(actual,predictions))
print(classification_report(actual,predictions))

[[58 18 11  2  2]
 [18 44 32  4  1]
 [11 24 65 20  4]
 [ 4  6 27 23 30]
 [ 7  0 12 21 56]]
              precision    recall  f1-score   support

           0       0.59      0.64      0.61        91
           1       0.48      0.44      0.46        99
           2       0.44      0.52      0.48       124
           3       0.33      0.26      0.29        90
           4       0.60      0.58      0.59        96

    accuracy                           0.49       500
   macro avg       0.49      0.49      0.49       500
weighted avg       0.49      0.49      0.49       500



In [10]:
def nearby_metric(predictions,actual):
    count=0
    for p,a in zip(predictions,actual):
        if (abs(p-a)<=1):
            count+=1
    return count/len(predictions)

nearby_metric(predictions,actual)

0.872

In [11]:
with open(b'metric_dist.pkl','rb') as f:
    new_dataset=pickle.load(f)

In [12]:
print(score(predictions,new_dataset))

155870.06671238854
